In [29]:
from typing import Any, Dict
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(temperature=0.1, streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()],)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self,examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
        
    def select_examples(self,input_variables):
        from random import choice
        return [choice(self.examples)]
    
# chat.predict("What do you know about France?")


In [33]:
example_prompt = PromptTemplate.from_template("Human:{country} \nAI:{answer}")
                                              
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
                                            
prompt.format(country="Brazil")

# chain = prompt | chat

# chain.invoke({
#     "country": "Germany"
# })

'Human:Greece \nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'